<a href="https://colab.research.google.com/github/fquinterov/TESEA/blob/main/scripts/reviewer_certificate_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reviewer Certificate Generator
---
Code by Fernando Quintero, Journal Manager

TESEA, Transactions on Energy Systems and Engineering Applications

In [1]:
!pip install --quiet pandas fpdf gspread oauth2client
!pip install --quiet python-docx
!pip install --quiet reportlab

  Preparing metadata (setup.py) ... done


In [113]:
from google.colab import auth
import gspread
import google.auth
import pandas as pd

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.units import mm
from reportlab.lib.colors import HexColor
from reportlab.lib.utils import simpleSplit
import pandas as pd
import os\


GITHUB_RAW_URL = "https://raw.githubusercontent.com/fquinterov/TESEA/main/assets/review_certificate_template.png"
os.system(f"wget -q {GITHUB_RAW_URL}")

0

In [114]:
auth.authenticate_user()

In [115]:
creds, _ = google.auth.default(scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"])
gc = gspread.authorize(creds)

from google.colab import userdata


SHEET_KEY = userdata.get('sheet_key')
sh = gc.open_by_key(SHEET_KEY)

ws = sh.sheet1
df = pd.DataFrame(ws.get_all_records())
df.head()

,full_name,affiliation,location,mail,title,volume,issue,date
0,Nasreen Mansoori,"Assistant Professor, Electrical Engineering De...","Silver Oak University, Gujarat, India",nasreenbanumansoori.ce@socet.edu.in,Advanced Power Quality Improvement Using Re-Li...,,,8/07/2025


In [116]:
def draw_certificate(data, out_path):
    # set up canvas
    c = canvas.Canvas(out_path, pagesize=landscape(A4))
    width, height = landscape(A4)

    # draw background template
    c.drawImage("/content/review_certificate_template.png", 0, 0, width=width, height=height)

    # subtitle / Awarded line
    c.setFont("Helvetica", 16)
    c.setFillColor(HexColor("#000000"))
    c.drawCentredString(width/2, height - 85*mm, "This is to certify that")

    # name
    c.setFont("Helvetica-Bold", 22)
    c.drawCentredString(width/2, height - 100*mm, data['full_name'])

    # body text with wrapping
    full_text = (
        f"has successfully completed the peer review of the manuscript titled "
        f"“{data['title']}” submitted to the Transactions on Energy Systems "
        f"and Engineering Applications."
    )
    max_width = 215 * mm
    wrapped = simpleSplit(full_text, "Helvetica", 14, max_width)

    text = c.beginText(30*mm, height - 115*mm)
    text.setFont("Helvetica", 16)
    text.setLeading(18)
    for line in wrapped:
        text.textLine(line)
    c.drawText(text)

    # date
    c.setFont("Helvetica", 16)
    c.drawCentredString(width/2, height - 145*mm, f"Date of Review Completion: “{data['date']}”")

    # editor info
    #c.setFont("Helvetica", 10)
    #c.drawString(30*mm, 35*mm, data['editor'])

    c.save()

target = 0
for idx, row in df.iterrows():
    if idx == target:
        safe_name = row['full_name'].replace(' ', '_')
        draw_certificate(row, f"cert_{safe_name}.pdf")
        break